In [1]:
# importing necessary modules.
import requests
import json
import pandas as pd
import numpy as np

In [2]:
# importing csv that contains the stock ticker symbols and names of the S&P 500.
sandp_df = pd.read_csv('../Resources/tickers_SandP_500.csv')
russell_2000_df = pd.read_csv('../Resources/tickers_russell_2000.csv')

In [3]:
# creating empty list.
ticker_symbols = []

# looping through above csv and appending stock ticker symbols to the empty list.
for ticker in sandp_df['Ticker']:
    ticker_symbols.append(ticker)
    
# looping through above csv and appending stock ticker symbols to the empty list.
# for ticker in russell_2000_df['Ticker']:
#     ticker_symbols.append(ticker)
    

# creating empty list.
names = []  

# looping through csv and appending names to the empty list.
for name in sandp_df['Name']:
    names.append(name)

# looping through csv and appending names to the empty list.
# for name in russell_2000_df['Name']:
#     names.append(name)

In [4]:
# creating/initializing an empty pandas dataframe.
dataframe2 = pd.DataFrame()

# creating loop that goes through the ticker_symbols and names lists created above.
for (ticker, name) in zip(ticker_symbols, names):
    
    # creating a for loop that goes through the last two entry's value of the json dictionaries from financialmodelingprep.    
    for i in range(-3,0):
        
        # setting up try and except statement to avoid lack of financialmodelingprep json data to ticker_symbol list.         
        try:
            
            # obtaining the json data for financial ratios from financialmodelingprep.          
            response = requests.get(f'https://financialmodelingprep.com/api/v3/financial-ratios/{ticker}').json()
            
            print(f'Processing:{ticker} {name}')
            
            # creating empty list named columns.             
            columns = []
            
            # appending 'symbol' to columns list.                
            columns.append('symbol')
            
            # appending 'name' to columns list.            
            columns.append('name')
        
            # appending 'date' to columns list.         
            columns.append('date')
            
            # creating empty list named data.            
            data = []
            
            # filtering the the financial ratios json dicionary data by years 2009 and 2010.            
            if response['ratios'][i]['date'].split('-')[0] == '2009' or response['ratios'][i]['date'].split('-')[0] == '2010' or response['ratios'][i]['date'].split('-')[0] == '2011':
                
                # creating another try except statement to avoid pitfalls.                
                try:
                    
                    # appending symbol to data list.
                    data.append(response['symbol'])
                   
                    # appending company name to data list.
                    data.append(name)
                    
                    # appending date to data list.
                    data.append(response['ratios'][i]['date'])
                    
                    # saving financial ratio dicionary to variable.
                    keys = response['ratios'][i]
                    
                    # looping through finanial ratio keys.                   
                    for key in keys:
                        
                        # avoiding date key.                       
                        if key != 'date':
                            
                            # looping through finanial ratio dictionary.                            
                            for x in keys[key]:
                                
                                #  appending financial ratio dict keys to columns list.                                
                                columns.append(x)
                                
                                # filtering financial ratio values that are blank.                 
                                if keys[key][x] == '':
                                    
                                    # appending 0 in the spot of nothing to the data list.                                     
                                    data.append(0)
                                
                                # continuing the filtering process, but with some values.         
                                else:
                                    
                                    # appending those financial ratio values to data list.                                 
                                    data.append(float(keys[key][x]))
                    
                    # creating a connection and obtaining financial growth statistics from financialmodelingprep.
                    response2 = requests.get(f'https://financialmodelingprep.com/api/v3/financial-statement-growth/{ticker}').json()
                    
                    # saving financial growth dict to variable.                  
                    keys2 = response2['growth'][i]
                    
                    # creating for loop to go through financial grow. 
                    for key in keys2:
                        
                        # avoiding the date key.                       
                        if key != 'date':
                            
                            # appending the financial growth keys to columns list.                            
                            columns.append(key)
                            
                            # filtering financial growth values that have nan values.
                            if keys2[key] == '':

                                # replacing nan values with 0.                                 
                                data.append(0)

                            # filtering financial growth values that are not nan.
                            else:

                                # appending those financial growth values to the data list.        
                                data.append(float(keys2[key]))

                    # creating a connection to get the company key metrics data from financialmodelingprep and putting in json form.
                    response3 = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}').json()
                    
                    # filtering share price values that are nan.
                    if response3['enterpriseValues'][i]['Stock Price'] == '':
                        
                        # setting returns variable to 0 based on no share price data.                    
                        returns = 0
                    
                    # filtering chare price based on having a value.
                    else:
                        
                        # using share price of 2009 and 2018/2019 to calculated percentage return over the time span.                        
                        returns = ((((float(response3['enterpriseValues'][0]['Stock Price']) + float(response3['enterpriseValues'][1]['Stock Price']))/2) - ((float(response3['enterpriseValues'][-2]['Stock Price']) + float(response3['enterpriseValues'][-1]['Stock Price']))/2))/((float(response3['enterpriseValues'][-2]['Stock Price']) + float(response3['enterpriseValues'][-1]['Stock Price']))/2))*100
                    
                    # creating a pandas dataframe based on the data and column lists developed through the above process.
                    dataframe = pd.DataFrame([data], columns=columns)
                    
                    # appending a share price return on investment to newly created dataframe.
                    dataframe['% return on investment'] = [returns]
                    
                    # appending the dataframe to the empty dataframe created at the very top.
                    dataframe2 = dataframe2.append(dataframe)
                
                # finishing try except statement.
                except:
                    
                    # telling program to move on and ignore pitfall.                    
                    pass
            
            # finishing date(2009,2010) if then statement created near the bottom of the top.         
            else:
                
                 # telling program to move on and ignore pitfall.
                pass
        
        # finishing the other try except statement.        
        except:
            
             # telling program to move on and ignore pitfall.
            pass

# grouping completed dataframe by symbol and name and finding the average of the columns, because used multiple dates.
dataframe2 = dataframe2.groupby(['symbol', 'name']).mean()

# viewing dataframe.
dataframe2

Processing:A Agilent Technologies, Inc.
Processing:A Agilent Technologies, Inc.
Processing:A Agilent Technologies, Inc.
Processing:AAL American Airlines Group, Inc.
Processing:AAL American Airlines Group, Inc.
Processing:AAL American Airlines Group, Inc.
Processing:AAP Advance Auto Parts, Inc.
Processing:AAP Advance Auto Parts, Inc.
Processing:AAP Advance Auto Parts, Inc.
Processing:AAPL Apple, Inc.
Processing:AAPL Apple, Inc.
Processing:AAPL Apple, Inc.
Processing:ABBV AbbVie, Inc.
Processing:ABBV AbbVie, Inc.
Processing:ABBV AbbVie, Inc.
Processing:ABC AmerisourceBergen Corp.
Processing:ABC AmerisourceBergen Corp.
Processing:ABC AmerisourceBergen Corp.
Processing:ABMD ABIOMED, Inc.
Processing:ABMD ABIOMED, Inc.
Processing:ABMD ABIOMED, Inc.
Processing:ABT Abbott Laboratories
Processing:ABT Abbott Laboratories
Processing:ABT Abbott Laboratories
Processing:ACN Accenture Plc
Processing:ACN Accenture Plc
Processing:ACN Accenture Plc
Processing:ADBE Adobe, Inc.
Processing:ADBE Adobe, Inc.

Processing:BRK.B Berkshire Hathaway, Inc.
Processing:BSX Boston Scientific Corp.
Processing:BSX Boston Scientific Corp.
Processing:BSX Boston Scientific Corp.
Processing:BWA BorgWarner, Inc.
Processing:BWA BorgWarner, Inc.
Processing:BWA BorgWarner, Inc.
Processing:BXP Boston Properties, Inc.
Processing:BXP Boston Properties, Inc.
Processing:BXP Boston Properties, Inc.
Processing:C Citigroup, Inc.
Processing:C Citigroup, Inc.
Processing:C Citigroup, Inc.
Processing:CAG Conagra Brands, Inc.
Processing:CAG Conagra Brands, Inc.
Processing:CAG Conagra Brands, Inc.
Processing:CAH Cardinal Health, Inc.
Processing:CAH Cardinal Health, Inc.
Processing:CAH Cardinal Health, Inc.
Processing:CAT Caterpillar, Inc.
Processing:CAT Caterpillar, Inc.
Processing:CAT Caterpillar, Inc.
Processing:CB Chubb Ltd.
Processing:CB Chubb Ltd.
Processing:CB Chubb Ltd.
Processing:CBOE Cboe Global Markets, Inc.
Processing:CBOE Cboe Global Markets, Inc.
Processing:CBOE Cboe Global Markets, Inc.
Processing:CBRE CBRE G

Processing:DXC DXC Technology Co.
Processing:DXC DXC Technology Co.
Processing:DXC DXC Technology Co.
Processing:EA Electronic Arts, Inc.
Processing:EA Electronic Arts, Inc.
Processing:EA Electronic Arts, Inc.
Processing:EBAY eBay, Inc.
Processing:EBAY eBay, Inc.
Processing:EBAY eBay, Inc.
Processing:ECL Ecolab, Inc.
Processing:ECL Ecolab, Inc.
Processing:ECL Ecolab, Inc.
Processing:ED Consolidated Edison, Inc.
Processing:ED Consolidated Edison, Inc.
Processing:ED Consolidated Edison, Inc.
Processing:EFX Equifax, Inc.
Processing:EFX Equifax, Inc.
Processing:EFX Equifax, Inc.
Processing:EIX Edison International
Processing:EIX Edison International
Processing:EIX Edison International
Processing:EL The Estée Lauder Companies, Inc.
Processing:EL The Estée Lauder Companies, Inc.
Processing:EL The Estée Lauder Companies, Inc.
Processing:EMN Eastman Chemical Co.
Processing:EMN Eastman Chemical Co.
Processing:EMN Eastman Chemical Co.
Processing:EMR Emerson Electric Co.
Processing:EMR Emerson El

Processing:HON Honeywell International, Inc.
Processing:HON Honeywell International, Inc.
Processing:HP Helmerich & Payne, Inc.
Processing:HP Helmerich & Payne, Inc.
Processing:HP Helmerich & Payne, Inc.
Processing:HPE Hewlett-Packard Enterprise Co.
Processing:HPE Hewlett-Packard Enterprise Co.
Processing:HPE Hewlett-Packard Enterprise Co.
Processing:HPQ HP, Inc.
Processing:HPQ HP, Inc.
Processing:HPQ HP, Inc.
Processing:HRB H&R Block, Inc.
Processing:HRB H&R Block, Inc.
Processing:HRB H&R Block, Inc.
Processing:HRL Hormel Foods Corp.
Processing:HRL Hormel Foods Corp.
Processing:HRL Hormel Foods Corp.
Processing:HSIC Henry Schein, Inc.
Processing:HSIC Henry Schein, Inc.
Processing:HSIC Henry Schein, Inc.
Processing:HST Host Hotels & Resorts, Inc.
Processing:HST Host Hotels & Resorts, Inc.
Processing:HST Host Hotels & Resorts, Inc.
Processing:HSY The Hershey Co.
Processing:HSY The Hershey Co.
Processing:HSY The Hershey Co.
Processing:HUM Humana, Inc.
Processing:HUM Humana, Inc.
Processi

Processing:MCHP Microchip Technology, Inc.
Processing:MCHP Microchip Technology, Inc.
Processing:MCHP Microchip Technology, Inc.
Processing:MCK McKesson Corp.
Processing:MCK McKesson Corp.
Processing:MCK McKesson Corp.
Processing:MCO Moody's Corp.
Processing:MCO Moody's Corp.
Processing:MCO Moody's Corp.
Processing:MDLZ Mondelez International, Inc.
Processing:MDLZ Mondelez International, Inc.
Processing:MDLZ Mondelez International, Inc.
Processing:MDT Medtronic Plc
Processing:MDT Medtronic Plc
Processing:MDT Medtronic Plc
Processing:MET MetLife, Inc.
Processing:MET MetLife, Inc.
Processing:MET MetLife, Inc.
Processing:MGM MGM Resorts International
Processing:MGM MGM Resorts International
Processing:MGM MGM Resorts International
Processing:MHK Mohawk Industries, Inc.
Processing:MHK Mohawk Industries, Inc.
Processing:MHK Mohawk Industries, Inc.
Processing:MKC McCormick & Co., Inc.
Processing:MKC McCormick & Co., Inc.
Processing:MKC McCormick & Co., Inc.
Processing:MKTX MarketAxess Holdin

Processing:PSA Public Storage
Processing:PSA Public Storage
Processing:PSA Public Storage
Processing:PSX Phillips 66
Processing:PSX Phillips 66
Processing:PSX Phillips 66
Processing:PVH PVH Corp.
Processing:PVH PVH Corp.
Processing:PVH PVH Corp.
Processing:PWR Quanta Services, Inc.
Processing:PWR Quanta Services, Inc.
Processing:PWR Quanta Services, Inc.
Processing:PXD Pioneer Natural Resources Co.
Processing:PXD Pioneer Natural Resources Co.
Processing:PXD Pioneer Natural Resources Co.
Processing:PYPL PayPal Holdings, Inc.
Processing:PYPL PayPal Holdings, Inc.
Processing:PYPL PayPal Holdings, Inc.
Processing:QCOM QUALCOMM, Inc.
Processing:QCOM QUALCOMM, Inc.
Processing:QCOM QUALCOMM, Inc.
Processing:QRVO Qorvo, Inc.
Processing:QRVO Qorvo, Inc.
Processing:QRVO Qorvo, Inc.
Processing:RCL Royal Caribbean Cruises Ltd.
Processing:RCL Royal Caribbean Cruises Ltd.
Processing:RCL Royal Caribbean Cruises Ltd.
Processing:RE Everest Re Group Ltd.
Processing:RE Everest Re Group Ltd.
Processing:RE

Processing:UPS United Parcel Service, Inc.
Processing:UPS United Parcel Service, Inc.
Processing:UPS United Parcel Service, Inc.
Processing:URI United Rentals, Inc.
Processing:URI United Rentals, Inc.
Processing:URI United Rentals, Inc.
Processing:USB U.S. Bancorp
Processing:USB U.S. Bancorp
Processing:USB U.S. Bancorp
Processing:UTX United Technologies Corp.
Processing:UTX United Technologies Corp.
Processing:UTX United Technologies Corp.
Processing:V Visa, Inc.
Processing:V Visa, Inc.
Processing:V Visa, Inc.
Processing:VAR Varian Medical Systems, Inc.
Processing:VAR Varian Medical Systems, Inc.
Processing:VAR Varian Medical Systems, Inc.
Processing:VFC VF Corp.
Processing:VFC VF Corp.
Processing:VFC VF Corp.
Processing:VIAB Viacom Inc.
Processing:VIAB Viacom Inc.
Processing:VIAB Viacom Inc.
Processing:VLO Valero Energy Corp.
Processing:VLO Valero Energy Corp.
Processing:VLO Valero Energy Corp.
Processing:VMC Vulcan Materials Co.
Processing:VMC Vulcan Materials Co.
Processing:VMC Vulc

,,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,receivablesTurnover,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,...,5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,% return on investment
symbol,name,,,,,,,,,,,,,,,,,,,,,
A,"Agilent Technologies, Inc.",2.413067,4.774367,1.451467,7.232900,7.218200,14.934300,11.489000,0.0,0.0,0.0,...,0.000000,0.000000,0.074267,0.135267,0.098067,0.212733,0.076567,-0.024767,0.023367,215.407785
AAL,"American Airlines Group, Inc.",0.000000,0.000000,0.100967,0.000000,27.970767,0.000000,2.242833,0.0,0.0,0.0,...,0.000000,0.000000,0.043367,0.055367,-0.017600,-0.305567,0.064167,0.000000,0.039567,492.976589
AAP,"Advance Auto Parts, Inc.",4.709967,5.153300,0.815500,14.602133,52.804800,9.598200,6.588500,0.0,0.0,0.0,...,0.000000,0.000000,0.140433,0.081267,0.072700,0.004433,0.094767,0.000000,0.066900,158.290852
AAPL,"Apple, Inc.",5.246733,4.813033,3.805867,17.821233,10.061767,15.681700,13.559600,0.0,0.0,0.0,...,0.000000,0.000000,0.743667,0.314033,0.481267,0.487467,0.000000,0.300633,0.270100,641.441749
ABBV,"AbbVie, Inc.",16.468700,0.000000,3.019000,10.483600,2.426100,9.212300,8.745400,0.0,0.0,0.0,...,0.000000,0.000000,0.065800,0.021350,-0.038200,0.000000,0.000000,0.255300,0.341850,127.493713
ABC,AmerisourceBergen Corp.,2.922867,80.500667,0.110033,13.785867,20.047100,9.950733,8.263900,0.0,0.0,0.0,...,0.577833,0.257033,0.031267,0.091033,0.070800,0.078367,0.049033,0.000000,0.005667,258.118955
ABMD,"ABIOMED, Inc.",4.351400,7.031950,4.975700,0.000000,6.432350,0.000000,148.439500,0.0,0.0,0.0,...,0.000000,0.000000,-0.001400,-0.280950,-0.015700,-0.078500,0.000000,0.026300,0.061400,2583.734150
ABT,Abbott Laboratories,1.674967,3.618333,1.431067,7.892567,4.375967,5.582533,4.767033,0.0,0.0,0.0,...,0.009600,0.104033,0.121567,0.060967,0.128900,0.124767,0.134067,-0.072067,-0.019767,167.453799
ACN,Accenture Plc,8.150000,8.709233,1.096367,13.746933,6.247667,8.854300,8.343967,0.0,0.0,0.0,...,0.081900,0.316867,0.030067,0.000000,0.087133,0.142800,0.520300,0.000000,0.060967,363.726903


In [6]:
dataframe2 = dataframe2.drop(columns=['priceCashFlowRatio', 'priceEarningsToGrowthRatio', 'priceSalesRatio', 'dividendYield', 'priceFairValue', 'operatingProfitMargin'])

In [7]:
# filtering the completed dataframe by creating a new dataframe that the does not have the rows associated with 0 in the % return on investment column.
dataframe2 = dataframe2[dataframe2['% return on investment'] != 0]

In [8]:
dataframe2.to_csv('sandp2.csv')